# Data loader experiments

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data import *
from local.notebook.showdoc import show_doc

In [ ]:
from torch.utils.data import IterableDataset, get_worker_info

In [ ]:
import string
letters = list(string.ascii_lowercase)

In [ ]:
bs = 4

In [ ]:
def twoepochs(d): print(' '.join(''.join(o) for _ in range(2) for o in d))

In [ ]:
@patch
def __len__(self:DataLoader):
    return len((self._index_sampler, self.dataset)[isinstance(self.dataset, IterableDataset)])

## patch \_MapDatasetFetcher 

In [ ]:
@patch
def fetch(self:torch.utils.data._utils.fetch._MapDatasetFetcher, idxs):
    ds = self.dataset
    if self.auto_collation:
        if hasattr(self.collate_fn,"get_batch"): return self.collate_fn.get_batch(ds, idxs)
        data = [ds[idx] for idx in idxs]
    else: data = ds[idxs]
    return self.collate_fn(data)

In [ ]:
class CustCollate:
    def __init__(self, cf=default_collate): self.cf=cf
    def __call__(self, *args, **kwargs): self.cf(*args, **kwargs)
    def get_batch(self, ds, idxs): return self([ds[idx] for idx in idxs])

In [ ]:
ds = string.ascii_lowercase
dl = DataLoader(ds, batch_size=4, num_workers=0)#, collate_fn=CustCollate())

In [ ]:
[''.join(o) for o in dl]

['abcd', 'efgh', 'ijkl', 'mnop', 'qrst', 'uvwx', 'yz']

## BatchDS

In [ ]:
class SleepyDS():
    def __init__(self,coll): self.coll=coll
    def __len__(self): return len(self.coll)
    def __getitem__(self,i): time.sleep(0.02); return self.coll[i]

In [ ]:
class BatchDS(IterableDataset):
    def __init__(self, ds ,bs=1, shuffle=False, sampler=None, batch_sampler=None, drop_last=False,
                sampler_cls=None, batch_sampler_cls=BatchSampler, get_batch=None, reset=None):
        self.ds,self.samp,self.rng,self.nw,self.offs = ds,batch_sampler,random.Random(),1,0
        if self.samp: return
        if not sampler: sampler = ifnone(sampler_cls, (SequentialSampler,RandomSampler)[shuffle])(ds)
        self.samp = batch_sampler_cls(sampler, bs, drop_last)
        if get_batch: self.get_batch = types.MethodType(get_batch,self)
        if reset: self.reset = types.MethodType(reset,self)
        
    def __len__(self): return len(self.samp)
    def reset(self): pass
    def get_batch(self, b): return [self.ds[j] for j in b]
    def get_batches(self, idxs): return map(self.get_batch, idxs)
    def __iter__(self):
        self.reset()
        torch.manual_seed(self.rng.randint(0,sys.maxsize))
        samps = list(enumerate(self.samp))
        idxs = (b for i,b in samps if i%self.nw==self.offs)
        return self.get_batches(idxs)

def _wif(worker_id):
    info = get_worker_info()
    ds = info.dataset
    ds.nw,ds.offs = info.num_workers,info.id
    ds.samp.sampler = copy.copy(ds.samp.sampler)

In [ ]:
def get_dl(bs=4, collate_fn=default_collate, num_workers=0, **kwargs):
    dsi = BatchDS(SleepyDS(string.ascii_lowercase), bs=4, **kwargs)
    return DataLoader(dsi, num_workers=num_workers, batch_size=None, worker_init_fn=_wif, collate_fn=collate_fn)

In [ ]:
dl = get_dl(bs=4, num_workers=4, shuffle=True)
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

dgbs txfm znae jyir cvpw koql uh dgbs txfm znae jyir cvpw koql uh


In [ ]:
dl = get_dl(bs=4, num_workers=4)
%time twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz
CPU times: user 28.1 ms, sys: 36.2 ms, total: 64.2 ms
Wall time: 392 ms


In [ ]:
dl = get_dl(bs=4, num_workers=0)
%time twoepochs(dl)
len(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz
CPU times: user 2.8 ms, sys: 1.16 ms, total: 3.96 ms
Wall time: 1.05 s


7

In [ ]:
dl = get_dl(bs=4, num_workers=4, drop_last=True)
twoepochs(dl)
len(dl)

abcd efgh ijkl mnop qrst uvwx abcd efgh ijkl mnop qrst uvwx


6

In [ ]:
dl = get_dl(bs=4, num_workers=0, shuffle=True)
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

yqmb iezk axoc wthf gdsl rvnu jp vrmg whob xfql dsup tcjz anke iy


In [ ]:
ds = SleepyDS(string.ascii_lowercase)
dl = get_dl(bs=4, num_workers=4, sampler=SequentialSampler(ds))
twoepochs(dl)

abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz


In [ ]:
dl = get_dl(num_workers=4, batch_sampler=BatchSampler(RandomSampler(ds), 8, False))
twoepochs(dl)
test_eq(len(set(sum(dl,[]))), 26)

sazcwien ktfxoqmu rljdygpb hv sazcwien ktfxoqmu rljdygpb hv


In [ ]:
def rev_collate(s): return default_collate(list(reversed(s)))
dl = get_dl(bs=4, num_workers=4, collate_fn=rev_collate)
twoepochs(dl)

dcba hgfe lkji ponm tsrq xwvu zy dcba hgfe lkji ponm tsrq xwvu zy


In [ ]:
def get_batch(self, b): return [f'{self.ds[j]}/' for j in b[:2]]
dl = get_dl(bs=4, num_workers=4, get_batch=get_batch)
twoepochs(dl)

a/b/ e/f/ i/j/ m/n/ q/r/ u/v/ y/z/ a/b/ e/f/ i/j/ m/n/ q/r/ u/v/ y/z/


## fin